# Getting stock latest OHLCV

In [1]:
import pandas as pd
import numpy as np
import requests
import holidays
import json

from datetime import datetime
from pandas.tseries.offsets import BDay
from datetime import date

symbol = 'ANA.MC'

## Determining dates and closing values

When running the pipeline, the sequence of events must consider what is the last working day from which we have data.

I've prepared two classes called

`last`

and

`closing`
    
to provide the last working day, from the day it is invoked, and the closing value, of a stock symbol.

In [2]:
class last:

    @staticmethod
    def split_date(datestr: str) -> (int, int, int):
        """Simply splits the string date passed in a list with YYYY, MM and DD"""
        year = int(datestr[:4])
        month = int(datestr[5:7])
        day = int(datestr[8:])
        return [year, month, day]

    @staticmethod
    def business_day(datetime_str: str = None, strformat: bool = True):
        """
        Returns the last business day.
        :params datetime_str: A date in format 'YYYY-MM-DD' from which to compute 
            what is the last business day.
        :params strformat: Whether to convert the returns value to string. Default YES.
            Otherwise, the value returned is a datetime object
        """
        if datetime_str is None:
            datetime_obj = pd.datetime.today()
        else:
            datetime_obj = datetime.strptime(datetime_str, '%Y-%m-%d')

        last_business_day = datetime_obj - BDay(1)
        if strformat:
            return last_business_day.strftime('%Y-%m-%d')
        else:
            return last_business_day.to_pydatetime()
    
    @staticmethod
    def working_day(today: str = None, country: str = 'ES', max_days_back: int = 10):
        """Find the last working day from the reference date passed in `today`.
        Bank holidays are searched in the country specified in the second arg

        :param today: The date from which to start searching back for a working day.
            Default will use today's date.
        :param country: The country to use for bank holidays.
        :param max_days_back: Max nr. of days to search back for a working day.

        :return: The first working day, non-bank holiday, back from reference date.
                 If cannot be found within the max nr. of days back, returns 'UNKNOWN'
        """
        if today is None:
            today = datetime.today().strftime('%Y-%m-%d')
        ref_date = today
        last_business_day_found = False
        loop_iters = 0
        while not last_business_day_found and loop_iters < max_days_back:
            last_day = last.business_day(ref_date)
            if date(*last.split_date(last_day)) in getattr(holidays, country)():
                ref_date = last_day
                loop_iters += 1
            else:
                last_business_day_found = True
        if loop_iters < max_days_back:
            return last_day
        return 'UNKNOWN'


In [3]:
last.working_day('2019-11-04')

'2019-10-31'

In [4]:
class closing:
    """
    Get the last closing values from the specified stock provider
    """
    
    def alpha_vantage(url='https://www.alphavantage.co', 
                      api_entry='/query?', 
                      api_key='HF9S3IZSBKSKHPV3',
                      **kwargs):
        endpoint = url + api_entry
        arguments = '&'.join('{}={}'.format(key, value) for key, value in kwargs.items())
        arguments += '&apikey={}'.format(api_key)
        endpoint = endpoint+arguments
        response = requests.get(endpoint).json()
        stock_closing = dict()
        for old_key in response['Global Quote'].keys():
            stock_closing[old_key[4:]] = response['Global Quote'][old_key]
        return stock_closing
    
    def world_trading_data(url='https://api.worldtradingdata.com',
                          api_entry='/api/v1/history_multi_single_day?',
                          api_key='WzDx1XnfvdGGBbfmAGwBuZBBnWKhMCIqbbVlmVKK3jFIvSLVphldCZPqOy2l',
                          **kwargs):
        endpoint = url + api_entry
        arguments = '&'.join('{}={}'.format(key, value) for key, value in kwargs.items())
        arguments += '&api_token={}'.format(api_key)
        endpoint = endpoint+arguments
        response = requests.get(endpoint).json()
        return response

To get the stock data, we call `closing.alpha_vantage` which is the only one that is working at the moment. The result is a dictionary with a set of fields that will contain all the required information.

In [5]:
stock_data = closing.alpha_vantage(function='GLOBAL_QUOTE', symbol=symbol)
stock_data

{'symbol': 'ANA.MC',
 'open': '92.1500',
 'high': '96.3000',
 'low': '91.8000',
 'price': '94.6000',
 'volume': '130028',
 'latest trading day': '2019-11-08',
 'previous close': '92.7500',
 'change': '1.8500',
 'change percent': '1.9946%'}

Check if the data retrieved corresponds to the last working day. If so, I can obtain the OHLCV info, and dump it to the proper file(s)...

`stock_data['latest trading day'] == last.working_day()`

### Pipeline stage

Make a bundle to be executed from monday to friday that will:

1. Guess what is the last day it must retrieve data for
2. Call the stock info provider to retrieve that data
3. append that data to the OHLCV file

Given the dependencies outlined above, this piece of code should be called like this:

`latest_closing --symbol ANA.MC --file data/acciona_2019.csv`

In [36]:
def csv_row(stock_data: dict) -> str:
    # Copy the original repsonse from the provider to preserve it
    sd = stock_data.copy()
    for v in sd.keys():
        sd[v] = [sd[v]]

    # Create a dataframe from it
    my_columns = ['latest trading day', 'open', 'high', 'low', 'price', 'volume']
    latest_ohlcv = pd.DataFrame.from_dict(sd)
    latest_ohlcv = latest_ohlcv[my_columns].copy(deep=True)
    latest_ohlcv.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']

    # reduce the precision to two decimals, only.
    def f(x):
        if '.' in x:
            return x[0:x.index('.')+3]
        else:
            return x

    row = list(map(f, list(latest_ohlcv.values[0])))
    row = ','.join(map(str, row)) + '\n'
    return row

In [32]:
symbol = 'ANA.MC'

# Retrieve the latest stock info about the symbol and check if dates match
last_day = last.working_day()
stock_data = closing.alpha_vantage(function='GLOBAL_QUOTE', symbol=symbol)
if stock_data['latest trading day'] != last.working_day():
    msg = 'Latest stock DATE doesn\'t match last working day\n'
    msg +='  {} != {}'.format(stock_data['latest trading day'],
                              last.working_day())
    raise ValueError(msg)

row = csv_row(stock_data)
with open('/Users/renero/trader/data/prueba2019.csv','a') as fd:
    fd.write(row)
fd.close()